<a href="https://colab.research.google.com/github/Pedro0504/demo-repo/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-31 07:39:19--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  83.7MB/s    in 0.3s    

2024-12-31 07:39:19 (83.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [41]:
# add your code here - consider creating a new cell for each section of code
# we have to merge the datasets
df_ratings.dropna(how= 'any')

df_merge = df_ratings.merge(df_books, on='isbn', how = 'left')

# we need the value counts of the user and isbn columns
vcu = df_merge['user'].value_counts()
vci = df_merge['isbn'].value_counts()
# with those values we can filter the user with less than 200 ratings,
# and the isbn with less than 100 entries

vcu = vcu[vcu >= 200].index
vci = vci[vci >= 100].index

# we use a mask to filter the values

df_merge = df_merge[df_merge['user'].isin(vcu)]
df_merge = df_merge[df_merge['isbn'].isin(vci)]

# we have to eliminate the duplicates and null values

df_merge.drop_duplicates(subset = ['title', 'user'], inplace = True)
df_merge.dropna(how='any')
df_merge.reset_index(drop = True, inplace = True)
df_merge.isna().sum()
df_merge.dropna(how='any')
df_merge.reset_index(drop = True, inplace = True)
df_pivot = df_merge.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)

df_matrix = csr_matrix(df_pivot.values)

knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', p = 2)
knn.fit(df_matrix)




<ipython-input-41-27f3bd0cc02b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge.drop_duplicates(subset = ['title', 'user'], inplace = True)


NearestNeighbors(algorithm='brute', metric='cosine')

In [42]:
# function to return recommended books - this will be tested

def get_recommends(book = ""):
    recommended_books = [book, []]
    if book not in df_pivot.index:
        return ["El libro no está en la base de datos."]

    distance, indices = knn.kneighbors([df_pivot.loc[book]], 6, return_distance = True)

    rec_binfo = df_pivot.iloc[np.flip(indices[0])[:-1]].index.to_list()
    rec_distance = list(np.flip(distance[0])[:-1])
    for r in zip(rec_binfo, rec_distance):
        recommended_books[1].append(list(r))

    return recommended_books



In [43]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
  ['Interview with the Vampire', 0.73450685],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]

In [44]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
